In [1]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import asyncio
from typing import Optional, Dict, Any, List
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client
from contextlib import asynccontextmanager

class SimpleMCPClient:
    def __init__(self):
        self.session: Optional[ClientSession] = None
        self.tools = None
    
    @asynccontextmanager
    async def connect(self, server_url: str = "http://localhost:8000/sse"):
        """创建SSE连接的上下文管理器"""
        try:
            async with sse_client(server_url) as streams:
                read_stream, write_stream = streams
                async with ClientSession(read_stream, write_stream) as session:
                    self.session = session
                    await self.session.initialize()
                    result = await self.session.list_tools()
                    self.tools = result.tools
                    yield self
        finally:
            self.session = None
            self.tools = None

    async def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Any:
        """调用指定的工具"""
        if not self.session:
            raise RuntimeError("Not connected to server")
        
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            raise ValueError(f"Tool {tool_name} not found")
        
        return await self.session.call_tool(tool, arguments)
    
# 在notebook中使用的辅助函数
async def run_in_notebook(coroutine):
    """在notebook中运行异步代码的辅助函数"""
    return await coroutine

In [10]:
# 创建客户端实例
client = SimpleMCPClient()

# 方式1：使用异步上下文管理器
async with client.connect() as c:
    # 列出可用工具
    print("Available tools:", [t.name for t in c.tools])
    print(c.tools)
    # 调用工具示例
    # result = await c.call_tool("tool_name", {"arg": "value"})
    # print("Result:", result)

# # 方式2：使用辅助函数在notebook中运行
# async def main():
#     async with client.connect() as c:
#         tools = c.tools
#         print("Available tools:", [t.name for t in tools])
        
#         # # 调用工具示例
#         # result = await c.call_tool("tool_name", {"arg": "value"})
#         # return result

# # 在notebook中运行
# result = await run_in_notebook(main())

Available tools: ['create-knowledge-folder', 'update-knowledge-folder', 'move-knowledge-folder', 'delete-knowledge-folder', 'get-knowledge-directory', 'get-knowledge-capacity', 'get-knowledge-file-list', 'get-knowledge-file-tags', 'add-knowledge-file-tag', 'remove-knowledge-file-tag', 'get-knowledge-note', 'save-knowledge-note', 'get-scholar-info', 'get-scholar-coauthors', 'search-scholars', 'batch-get-scholars', 'get-scholar-papers', 'get-follow-list', 'get-subscription-list', 'search-papers-normal', 'search-papers-enhanced', 'search-papers-pro-v1', 'search-papers-pro-v2']
[Tool(name='create-knowledge-folder', description='在知识库中创建新文件夹', inputSchema={'type': 'object', 'properties': {'parent_id': {'type': 'integer', 'description': '父文件夹ID'}, 'folder_name': {'type': 'string', 'description': '文件夹名称'}}, 'required': ['parent_id', 'folder_name']}, annotations=None), Tool(name='update-knowledge-folder', description='更新知识库中的文件夹名称', inputSchema={'type': 'object', 'properties': {'folder_id': {'t

In [1]:
content = None
if content:
    print(content)
else:
    print("content is None")

content is None


In [5]:
d = {}
# d["scholar_ids"]
c = d.get("scholar_ids")
print(c)
if c is None:
    print("c is None")
else:
    print("c is not None")


None
c is None


# debug 测试 tool call none的问题

In [ ]:
from dotenv import load_dotenv
from openai import AzureOpenAI
import os
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# 定义工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市名称"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# 发送请求
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL"),
    messages=[
        {"role": "user", "content": "北京今天天气怎么样？"}
    ],
    # tools=tools,
    # tool_choice="auto"
)



In [7]:
# 方式1：使用 json 模块格式化打印
import json

# 将response对象转换为字典并格式化打印
print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))


{
  "id": "chatcmpl-BbgMHRcSFyEA0RtVqdR2L3plcGLtp",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "截至我的知识截止日期是2023年10月，我无法提供今日的实时天气情况。建议您使用以下方式获取北京的实时天气信息：\n\n1. **搜索引擎查询**：在百度或谷歌搜索“北京天气”即可获得最新天气情况。\n2. **天气应用**：通过手机上的天气应用（如墨迹天气、微软天气、Weather.com等）查看实时天气。\n3. **微信公众号或小程序**：例如“中国天气”官方公众号或小程序。\n\n通常提供的信息包括气温、降水、空气质量和风速等，祝您安排好日常活动！",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],


In [8]:
print(response)

ChatCompletion(id='chatcmpl-BbgMHRcSFyEA0RtVqdR2L3plcGLtp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='截至我的知识截止日期是2023年10月，我无法提供今日的实时天气情况。建议您使用以下方式获取北京的实时天气信息：\n\n1. **搜索引擎查询**：在百度或谷歌搜索“北京天气”即可获得最新天气情况。\n2. **天气应用**：通过手机上的天气应用（如墨迹天气、微软天气、Weather.com等）查看实时天气。\n3. **微信公众号或小程序**：例如“中国天气”官方公众号或小程序。\n\n通常提供的信息包括气温、降水、空气质量和风速等，祝您安排好日常活动！', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1748320661, model='gpt-4o-2024-11-20', object='chat.completion', service_tier=None, system_fingerprint='fp_ee1d74bde0', usage=CompletionUsage(completion_tokens=132, prompt_tokens=13, total_tokens=145, completion_tokens_details=CompletionTokensDetails(accepted_predic

In [ ]:

# 示例响应1 - 调用工具的情况
"""
{
  "id": "chatcmpl-BbgKwNr2saETFAEHm30Qavz6b826d",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_YiuP80DGWyF4ggjzaetKR6KE",
            "function": {
              "arguments": "{\"location\":\"北京\"}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1748320578,
  "model": "gpt-4o-2024-11-20",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_ee1d74bde0",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 52,
    "total_tokens": 67,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ]
}

"""

# 示例响应2 - 直接回答的情况
"""
{
  "id": "chatcmpl-BbgMHRcSFyEA0RtVqdR2L3plcGLtp",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "截至我的知识截止日期是2023年10月，我无法提供今日的实时天气情况。建议您使用以下方式获取北京的实时天气信息：\n\n1. **搜索引擎查询**：在百度或谷歌搜索“北京天气”即可获得最新天气情况。\n2. **天气应用**：通过手机上的天气应用（如墨迹天气、微软天气、Weather.com等）查看实时天气。\n3. **微信公众号或小程序**：例如“中国天气”官方公众号或小程序。\n\n通常提供的信息包括气温、降水、空气质量和风速等，祝您安排好日常活动！",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1748320661,
  "model": "gpt-4o-2024-11-20",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_ee1d74bde0",
  "usage": {
    "completion_tokens": 132,
    "prompt_tokens": 13,
    "total_tokens": 145,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ]
}

"""

# debug 测试 连续两条user message 
 

测试结果发现 连续两条user消息是没有问题的

In [9]:
from dotenv import load_dotenv
from openai import AzureOpenAI
import os
import json

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# 定义工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_papers",
            "description": "搜索学术论文",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "搜索关键词"
                    },
                    "year_start": {
                        "type": "integer",
                        "description": "起始年份"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

def simulate_conversation():
    messages = []
    
    # 步骤1: 初始用户查询
    messages.append({"role": "user", "content": "分析张林峰的研究工作"})
    
    # 第一轮：助手使用工具
    response1 = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL"),
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    print("第一轮助手回复：")
    print(json.dumps(response1.model_dump(), indent=2))
    
    # 模拟工具返回结果
    if response1.choices[0].message.tool_calls:
        tool_result = {
            "papers": [
                {"title": "DeePMD-kit", "year": 2023},
                {"title": "Machine learning potentials", "year": 2022}
            ]
        }
        messages.append(response1.choices[0].message.model_dump())
        messages.append({
            "role": "tool",
            "tool_call_id": response1.choices[0].message.tool_calls[0].id,
            "name": "search_papers",
            "content": json.dumps(tool_result)
        })
    
    # 步骤2: 添加反思提示
    reflection_prompt = {
        "role": "user",
        "content": (
            "Based on the tool results, please analyze:\n"
            "1. What have we learned?\n"
            "2. Do we need more information?\n"
            "3. Are we ready for a Final Answer?\n"
        )
    }
    messages.append(reflection_prompt)
    
    # 获取带有 Final Answer 但表明需要更多信息的响应
    response2 = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL"),
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    print("\n第二轮助手回复（包含 Final Answer 但需要更多信息）：")
    print(json.dumps(response2.model_dump(), indent=2))
    
    # 步骤3: 测试连续的用户提示
    messages.append(response2.choices[0].message.model_dump())
    messages.append({
        "role": "user",
        "content": (
            "I notice that we still need more information or analysis. "
            "Please continue with the necessary tool calls or analysis "
            "before providing the final answer."
        )
    })
    
    # 立即跟随另一个用户提示（这里模拟了问题场景）
    messages.append({
        "role": "user",
        "content": (
            "Based on the previous results, please analyze:\n"
            "1. What have we learned?\n"
            "2. Do we need more information?\n"
            "3. Are we ready for a Final Answer?\n"
        )
    })
    
    # 获取最终响应
    response3 = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL"),
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    print("\n第三轮助手回复（连续用户提示后）：")
    print(json.dumps(response3.model_dump(), indent=2))

# 运行测试
simulate_conversation()

第一轮助手回复：
{
  "id": "chatcmpl-Bbh4nOofnqop0B4xOxdRjETHZuLOo",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_yl5GLe0Weaad4TH1U0E7TxMp",
            "function": {
              "arguments": "{\"query\":\"\u5f20\u6797\u5cf0\"}",
              "name": "search_papers"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1748323421,
  "model": "gpt-4o-2024-11-20",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_ee1d74bde0",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 65,
    "total_tokens": 83,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
   